# Загрузка библиотек

In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset

import numpy as np
import math
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

# Подготовка данных

In [2]:
ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

# Путь к папке с данными
dataset_folder = f'/home/cv_learning/CV/CV-8+9. Генерация и перенос стиля/Celeb/CelebA/Img/img_align_celeba'

In [3]:
class CelebADataset(Dataset):
  def __init__(self, root_dir, transform=None):
    """
    Args:
      root_dir (string): Directory with all the images
      transform (callable, optional): transform to be applied to each image sample
    """
    # Считываем название файлов

    self.root_dir = root_dir
    self.transform = transform 
    self.image_names = os.listdir(root_dir)

  def __len__(self): 
    return len(self.image_names)

  def __getitem__(self, idx):
    # Получаем путь к файлу
    img_path = os.path.join(self.root_dir, self.image_names[idx])
    # Загружаем изображение
    img = Image.open(img_path).convert('RGB')
    # Применяем преобразования
    if self.transform:
      img = self.transform(img)

    return img

## Загружаем датасет 
# Путь к директории со всеми изображениями
# Фиксипуем размер изображения
image_size = 64
# Преоьразования
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
      mean=[0.5, 0.5, 0.5],
      std=[0.5, 0.5, 0.5])
])



celeba_dataset = CelebADataset(dataset_folder, transform)

batch_size = 128
num_workers = 0 if device.type == 'cuda' else 2
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = DataLoader(
  celeba_dataset,
  batch_size=batch_size,
  num_workers=num_workers,
  pin_memory=pin_memory,
  shuffle=True
)

# Модель

In [4]:
n_epochs = 2
lr = 0.0002
b1, b2 = 0.5, 0.999
channels = 3
latent_dim = 10
sample_interval = 400

img_shape = (channels, image_size, image_size)

cuda = True if torch.cuda.is_available() else False


def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), latent_dim))))
    z = sampled_z * std + mu
    return z


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.mu = nn.Linear(512, latent_dim)
        self.logvar = nn.Linear(512, latent_dim)

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        x = self.model(img_flat)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity


# Функция потерь
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

# Инициализируем генератор и дискриминатор
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()


# Оптимизаторы
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "data/%d.png" % batches_done, nrow=n_row, normalize=True)

# Обучение модели 

In [6]:
for epoch in range(n_epochs):
    for i, imgs in enumerate(celeba_dataloader):

        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Генератор
        # -----------------

        optimizer_G.zero_grad()

        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)

        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Дискриминатор
        # ---------------------

        optimizer_D.zero_grad()

        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(celeba_dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(celeba_dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

[Epoch 0/2] [Batch 0/1583] [D loss: 0.690496] [G loss: 0.555650]
[Epoch 0/2] [Batch 1/1583] [D loss: 0.682476] [G loss: 0.551544]
[Epoch 0/2] [Batch 2/1583] [D loss: 0.675199] [G loss: 0.511538]
[Epoch 0/2] [Batch 3/1583] [D loss: 0.654594] [G loss: 0.511390]
[Epoch 0/2] [Batch 4/1583] [D loss: 0.639780] [G loss: 0.502827]
[Epoch 0/2] [Batch 5/1583] [D loss: 0.634238] [G loss: 0.476861]
[Epoch 0/2] [Batch 6/1583] [D loss: 0.632450] [G loss: 0.482778]
[Epoch 0/2] [Batch 7/1583] [D loss: 0.623898] [G loss: 0.462934]
[Epoch 0/2] [Batch 8/1583] [D loss: 0.599057] [G loss: 0.463240]
[Epoch 0/2] [Batch 9/1583] [D loss: 0.589076] [G loss: 0.431849]
[Epoch 0/2] [Batch 10/1583] [D loss: 0.578618] [G loss: 0.427873]
[Epoch 0/2] [Batch 11/1583] [D loss: 0.565402] [G loss: 0.403671]
[Epoch 0/2] [Batch 12/1583] [D loss: 0.541434] [G loss: 0.407199]
[Epoch 0/2] [Batch 13/1583] [D loss: 0.525832] [G loss: 0.383847]
[Epoch 0/2] [Batch 14/1583] [D loss: 0.518316] [G loss: 0.370833]
[Epoch 0/2] [Batch 1